In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from matplotlib.patches import Arc
import numpy as np
from math import sqrt


In [ ]:
class Ring:
    """
    Class representing a Cherenkov ring
    
    Unit radius, centred at (x, y) provided to the ctor

    """

    _x = 0.0
    _y = 0.0

    # Angle between the horizontal and the line joining the centre to the origin
    _alpha = 0.0

    # Distance to the origin
    _d = 0.0
    
    # Radius
    _radius = 1.0

    def __init__(self, x, y):
        self._x = x
        self._y = y
        self._alpha = np.arctan2(y, x)
        self._d = sqrt(x ** 2 + y ** 2)

    def _r(self, phi):
        """
        Returns the distance from the origin to the edge of the ring, looking along a line at an angle phi
        from the horizontal.

        phi in rad

        """
        # Our distance is the positive solution to the quadratic equation r^2 + br + c = 0
        # This just comes out of some geometry if you draw the triangles
        b = -2 * self._d * np.cos(phi - self._alpha)
        c = (self._d * self._d) - 1

        return (-b + sqrt(b * b - 4 * c)) / 2

    def misalignment(self, n):
        """
        Returns an array of n distances from the origin to the edge of the ring, equally spaced in angle
        between 0 and 2 pi

        Returns also an array of n array equally spaced between 0 and 2pi

        """
        angles = np.linspace(0, 2 * np.pi)

        # Could make this faster, don't need to
        return np.array([self._r(angle) for angle in angles]), angles

    def centre(self):
        return self._x, self._y
    
    def boundary(self, n):
        """
        Returns two n-length arrays (x, y) of x and y co-ordinates of the ring edges
        
        Not guaranteed to start at any particular angle - i.e. 
        
        """
        angles = np.linspace(0, 2*np.pi)
        return np.array([self._x + np.cos(x) for x in angles]), np.array([self._y + np.sin(x) for x in angles])

    def move(self, x, y):
        """
        Move this ring to be centred on (x, y)

        """
        self.__init__(x, y)
        

In [ ]:
def plot_ring(My_Ring, n, figsize=(9.0, 3.0)):
    """
    Create an interactive plot of a Cherenkov ring
    
    n points around the circle (higher => higher resolution, but slower to update)
    
    """
    distances, angles = My_Ring.misalignment(n)
    
    # Create some empty plots
    fig, _ = plt.subplots(figsize=figsize)
    ax = plt.subplot2grid((1, 3), (0, 0))
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax2 = plt.subplot2grid((1, 3), (0, 1), colspan=2)
    ax.axis("off")
    ax2.set_ylim(0, 2)
    ax2.set_yticks([])

    # Find where the centre and edge of the ring are
    circle_x, circle_y = My_Ring.boundary(n)
    orig_x, orig_y = My_Ring.centre()
    
    # Plot the Cherenkov Ring
    ring, = ax.plot(circle_x, circle_y, "k--")
    
    # Plot (and label) points representing the tracking spot and the centre of the Cherenkov ring 
    ax.plot(0, 0, "r.", label="Tracking Spot")
    ring_centre, = ax.plot(orig_x, orig_y, "bx", label="Cherenkov Ring Centre")
    ax.legend(loc=(-0.5, 0.9))

    # Draw a horizontal line from the tracking spot
    horizontal, = ax.plot((0, circle_x[0]), (0, 0), "k")

    # Mark an angle to illustrate what we're doing: about 108 degrees (when the ring is centred)
    angled_line, = ax.plot((0, circle_x[n // 5]), (0, circle_y[n // 5]), "k")
    phi = np.arctan2(circle_y[n // 5] / 5, circle_x[n // 5] / 5)
    
    # Label the length and angle of the line from the tracking spot to the ring
    ax.text(0.1, 0.1, r"$\phi$")
    r_label = ax.text(-0.2 + circle_x[n // 5] / 2, circle_y[n // 5] / 2, "r")

    # Plot the distance from the tracking spot to the ring as a function of angle
    line, = ax2.plot(angles, distances)
    ax2.set_ylabel(r"$r\ (\phi)$")
    x_tick = np.linspace(0, 2, 5)
    ax2.set_xticks(x_tick * np.pi)
    ax2.set_xticklabels([fr"${x}\pi$" for x in x_tick])
    ax2.tick_params(direction="in")
    ax2.text(np.pi, 0.1, r"$\phi\ /rad$", horizontalalignment="center")

    # Create sliders that will allow us to move the Cherenkov ring around
    ax_x_slider = plt.axes([0.05, 0.1, 0.3, 0.03])
    ax_y_slider = plt.axes([0.05, 0.05, 0.3, 0.03])
    x_slider = Slider(ax_x_slider, "x", -1.0, 1.0, orig_x)
    y_slider = Slider(ax_y_slider, "y", -1.0, 1.0, orig_y)

    def update(val):
        """
        Move all the components of the plot according to the values of the sliders
        
        """
        # Find the new centre of the ring
        x = x_slider.val
        y = y_slider.val
        
        # Move the ring and its central dot
        My_Ring.move(x, y)
        ring_centre.set_data(x, y)     
        
        # Find the new locations of the Cherenkov ring and its distances from
        ring_coords = My_Ring.boundary(n)
        misalignment, _ = My_Ring.misalignment(n)
        
        # Update the geometric bits (horizontal line, line at an angle phi, etc.)
        horizontal.set_xdata((0, misalignment[0]))
        angled_line.set_data((0, ring_coords[0][n//5]), (0, ring_coords[1][n//5]))
        r_label.set_position((-0.2+ring_coords[0][n//5]/2, ring_coords[1][n//5]/2))
        
        # Update the distance from the tracking spot as a function of angle
        ring.set_data(*ring_coords)
            
        # Update the plot of distance vs angle
        line.set_ydata(misalignment)
    

    # When our slider changes, update the plot
    x_slider.on_changed(update)
    y_slider.on_changed(update)
    
    plt.show()

plot_ring(Ring(0.0, 0.0), 25)
